In [1]:
from __future__ import annotations

from typing import Iterable, Dict, Optional, Tuple, List, Set
import re

import pandas as pd
import geopandas as gpd
import fiona
from IPython.display import display

# === Percorsi (modifica se necessario) ===
TSV_PATH = "data/gadm1_nuts3_counties-gadm1_nuts3_counties - FB Social Connectedness Index - October 2021.tsv"
MAP_PATH = "data/gadm1_nuts3_counties_levels.csv"

# Geodati
NUTS_GEOJSON_PATH   = "data/NUTS_RG_60M_2016_4326_LEVL_3.geojson"
US_COUNTIES_PATH    = "data/us-county-boundaries.geojson"
GADM_GPKG_PATH      = "data/gadm_410.gpkg"   # layer tipico: "gadm_410"

# Parametri lettura SCI
SCI_COLS  = ["user_loc", "fr_loc", "scaled_sci"]
DTYPE_MAP = {"user_loc": "string", "fr_loc": "string", "scaled_sci": "float64"}

# ==========================
# Utilità base
# ==========================

def _normalize_location_code(s: pd.Series) -> pd.Series:
    """
    Normalizza i codici location: stringa, strip, upper (non tocca NaN).
    """
    return s.astype("string").str.strip().str.upper()


def _ensure_columns(df: pd.DataFrame, required: Iterable[str], where: str = "") -> None:
    """
    Verifica che il DataFrame contenga le colonne richieste.
    """
    missing = [c for c in required if c not in df.columns]
    if missing:
        loc = f" in {where}" if where else ""
        raise ValueError(f"Mancano colonne{loc}: {missing}")


def _preview(df: pd.DataFrame, name: str = "DataFrame", n: int = 5) -> None:
    """
    Stampa anteprima, schema e numero righe.
    """
    print(f"== {name} – prime righe ==")
    display(df.head(n))
    print("\nSchema:")
    print(df.dtypes)
    print(f"\nNumero righe: {len(df):,}")


# ==========================
# Lettura input SCI + mapping
# ==========================

def load_sci_tsv(
    path: str,
    sci_cols: Iterable[str] = ("user_loc", "fr_loc", "scaled_sci"),
    dtype_map: Optional[Dict[str, str]] = None,
    low_memory: bool = False
) -> pd.DataFrame:
    if dtype_map is None:
        dtype_map = {"user_loc": "string", "fr_loc": "string", "scaled_sci": "float64"}

    df = pd.read_csv(
        path,
        sep="\t",
        usecols=list(sci_cols),
        dtype=dtype_map,
        low_memory=low_memory
    )

    # normalizza codici
    if "user_loc" in df.columns:
        df["user_loc"] = _normalize_location_code(df["user_loc"])
    if "fr_loc" in df.columns:
        df["fr_loc"] = _normalize_location_code(df["fr_loc"])

    _ensure_columns(df, sci_cols, where="SCI TSV")
    return df


def load_levels_mapping(
    path: str,
    usecols: Iterable[str] = ("key", "level"),
    rename_map: Dict[str, str] = {"key": "location_code", "level": "level_type"},
) -> pd.DataFrame:
    df = pd.read_csv(path, usecols=list(usecols), dtype="string").rename(columns=rename_map)
    _ensure_columns(df, ["location_code", "level_type"], where="Mapping livelli")
    df["location_code"] = _normalize_location_code(df["location_code"])
    return df


def quick_read_inputs(
    tsv_path: str,
    map_path: str,
    sci_cols: Iterable[str] = ("user_loc", "fr_loc", "scaled_sci"),
    dtype_map: Optional[Dict[str, str]] = None
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df_sci = load_sci_tsv(tsv_path, sci_cols=sci_cols, dtype_map=dtype_map)
    _preview(df_sci, name="SCI (TSV)")
    df_map = load_levels_mapping(map_path)
    _preview(df_map, name="Mapping livelli")
    return df_sci, df_map

# Esecuzione lettura
df_sci, df_map = quick_read_inputs(TSV_PATH, MAP_PATH, sci_cols=SCI_COLS, dtype_map=DTYPE_MAP)

# Facoltativo: distribuzione tipi livello
print("\n== level_type value_counts ==")
if "level_type" in df_map:
    display(df_map['level_type'].value_counts())


# ==========================
# Metriche di copertura
# ==========================

def get_unique_location_codes(df_sci: pd.DataFrame) -> pd.Series:
    _ensure_columns(df_sci, ["user_loc", "fr_loc"], where="SCI")
    u = _normalize_location_code(df_sci["user_loc"]) 
    v = _normalize_location_code(df_sci["fr_loc"]) 
    return pd.Index(u).append(pd.Index(v)).astype("string").unique()


def get_mapped_codes(df_map: pd.DataFrame) -> pd.Series:
    _ensure_columns(df_map, ["location_code"], where="Mapping livelli")
    return _normalize_location_code(df_map["location_code"]).dropna().unique()


def compute_node_coverage(df_sci: pd.DataFrame, df_map: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, float]]:
    sci_codes = pd.Series(get_unique_location_codes(df_sci), name="location_code")
    map_codes = pd.Series(get_mapped_codes(df_map), name="location_code")

    total_codes = sci_codes.size
    mapped_mask = sci_codes.isin(set(map_codes))
    mapped_count = int(mapped_mask.sum())
    unmapped = (
        sci_codes[~mapped_mask].to_frame().drop_duplicates()
        .sort_values("location_code").reset_index(drop=True)
    )

    summary = {
        "total_unique_codes": int(total_codes),
        "mapped_unique_codes": int(mapped_count),
        "unmapped_unique_codes": int(total_codes - mapped_count),
        "node_coverage_pct": (mapped_count / total_codes * 100.0) if total_codes else 0.0,
    }
    return unmapped, summary


def compute_edge_coverage(df_sci: pd.DataFrame, df_map: pd.DataFrame) -> Dict[str, float]:
    _ensure_columns(df_sci, ["user_loc", "fr_loc"], where="SCI")
    mapped_set = set(get_mapped_codes(df_map))

    u = _normalize_location_code(df_sci["user_loc"]) 
    v = _normalize_location_code(df_sci["fr_loc"]) 

    both_mapped_mask = u.isin(mapped_set) & v.isin(mapped_set)
    total_rows = int(len(df_sci))
    valid_rows = int(both_mapped_mask.sum())

    return {
        "total_rows": total_rows,
        "valid_rows_both_mapped": valid_rows,
        "edge_coverage_pct": (valid_rows / total_rows * 100.0) if total_rows else 0.0,
    }


def compute_country_coverage(
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    iso_col: str = "country_ISO3",
    keep_only_intra_country: bool = True
) -> Optional[pd.DataFrame]:
    if iso_col not in df_map.columns:
        print(f"[AVVISO] Colonna '{iso_col}' assente nel mapping: copertura per paese non calcolata.")
        return None

    _ensure_columns(df_sci, ["user_loc", "fr_loc"], where="SCI")

    df_map_local = df_map[["location_code", iso_col]].copy()
    df_map_local["location_code"] = _normalize_location_code(df_map_local["location_code"])
    code2iso = (
        df_map_local.dropna().drop_duplicates("location_code")
        .set_index("location_code")[iso_col]
    )

    sci = df_sci[["user_loc", "fr_loc"]].copy()
    sci["user_loc"] = _normalize_location_code(sci["user_loc"])
    sci["fr_loc"]   = _normalize_location_code(sci["fr_loc"])

    sci["iso_from"] = sci["user_loc"].map(code2iso)
    sci["iso_to"]   = sci["fr_loc"].map(code2iso)

    sci_valid = sci.dropna(subset=["iso_from", "iso_to"]).copy()
    if keep_only_intra_country:
        sci_valid = sci_valid[sci_valid["iso_from"] == sci_valid["iso_to"]].copy()

    edges_by_country = (
        sci_valid.groupby("iso_from", as_index=False)
        .agg(edges=("user_loc", "size")).rename(columns={"iso_from": iso_col})
    )

    nodes_from = sci_valid[["iso_from", "user_loc"]].rename(columns={"iso_from": iso_col, "user_loc": "loc"})
    nodes_to   = sci_valid[["iso_to", "fr_loc"]].rename(columns={"iso_to": iso_col, "fr_loc": "loc"})
    nodes_all  = pd.concat([nodes_from, nodes_to], ignore_index=True).drop_duplicates()

    nodes_by_country = nodes_all.groupby(iso_col, as_index=False).agg(nodes=("loc", "nunique"))

    country_cov = edges_by_country.merge(nodes_by_country, on=iso_col, how="outer").fillna(0)
    country_cov["edges"] = country_cov["edges"].astype(int)
    country_cov["nodes"] = country_cov["nodes"].astype(int)

    total_edges = int(country_cov["edges"].sum()) if len(country_cov) else 0
    country_cov["edges_pct"] = (100.0 * country_cov["edges"] / total_edges) if total_edges else 0.0

    return country_cov.sort_values(["edges", "nodes"], ascending=False).reset_index(drop=True)


def coverage_report(df_sci: pd.DataFrame, df_map: pd.DataFrame, top_n: int = 10) -> None:
    unmapped_nodes, node_summary = compute_node_coverage(df_sci, df_map)
    print("== Copertura NODI ==")
    for k, v in node_summary.items():
        print(f"- {k}: {v}")
    if len(unmapped_nodes) > 0:
        print(f"\nEsempi codici NON mappati ({min(10, len(unmapped_nodes))}):")
        display(unmapped_nodes.head(10))

    edge_summary = compute_edge_coverage(df_sci, df_map)
    print("\n== Copertura ARCHI ==")
    for k, v in edge_summary.items():
        print(f"- {k}: {v}")

    country_cov = compute_country_coverage(df_sci, df_map)
    if country_cov is not None and len(country_cov):
        print("\n== Copertura PER PAESE (prime righe) ==")
        display(country_cov.head(top_n))
    else:
        print("\n== Copertura PER PAESE ==")
        print("Mapping senza 'country_ISO3' → se vuoi questa sezione, aggiungi la colonna a df_map.")

# Esecuzione report
coverage_report(df_sci, df_map)


# ==========================
# Funzioni geodati (con 'name' e dataset di origine)
# ==========================

def _representative_points(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.to_crs(4326)
    gdf["geometry"] = gdf["geometry"].representative_point()
    gdf["latitude"] = gdf.geometry.y
    gdf["longitude"] = gdf.geometry.x
    return gdf


def load_nuts3_points(nuts_geojson_path: str, code_col: Optional[str] = None) -> pd.DataFrame:
    gdf = gpd.read_file(nuts_geojson_path)
    # Filtro LEVL=3
    levl_col = next((c for c in ["LEVL_CODE", "LEVL", "LEVEL"] if c in gdf.columns), None)
    if levl_col is not None:
        gdf = gdf[gdf[levl_col].astype(str).isin(["3", 3])].copy()

    # Codice e Nome
    candidates_code = [code_col] if code_col else [c for c in ["NUTS_ID", "nuts_id", "ID", "id"] if c in gdf.columns]
    if not candidates_code:
        raise ValueError("Non trovo una colonna codice per NUTS (es. 'NUTS_ID'). Passa code_col=...")
    cc = candidates_code[0]

    name_col = next((c for c in ["NUTS_NAME", "NAME_LATN", "NAME_ENGL", "NAME"] if c in gdf.columns), None)

    gdf = gdf.rename(columns={cc: "code", (name_col if name_col else cc): "name"})
    cols = ["code", "name", "geometry"] if name_col else ["code", "geometry"]
    gdf = _representative_points(gdf[cols].dropna(subset=["code"]))

    gdf["code"] = _normalize_location_code(gdf["code"])  # es. "AL034"
    if "name" not in gdf:
        gdf["name"] = pd.NA

    out = gdf[["code", "name", "latitude", "longitude"]].drop_duplicates("code")
    return out


def load_gadm2_points(gadm_gpkg_path: str, layer: str = "gadm_410", code_col: str = "GID_2") -> pd.DataFrame:
    gdf = gpd.read_file(gadm_gpkg_path, layer=layer)
    if code_col not in gdf.columns:
        raise ValueError(f"La colonna {code_col!r} non esiste nel layer {layer}. Colonne trovate: {list(gdf.columns)}")

    name_col = "NAME_2" if "NAME_2" in gdf.columns else None

    gdf2 = gdf[~gdf[code_col].isna()].copy().rename(columns={code_col: "code"})
    if name_col:
        gdf2 = gdf2.rename(columns={name_col: "name"})
    gdf2 = gdf2.to_crs(4326)
    gdf2["geometry"] = gdf2.geometry.representative_point()
    gdf2["latitude"] = gdf2.geometry.y
    gdf2["longitude"] = gdf2.geometry.x
    gdf2["code"] = gdf2["code"].astype("string").str.strip().str.upper()  # es. "AGO.4.7_1"
    if "name" not in gdf2:
        gdf2["name"] = pd.NA

    out = gdf2[["code", "name", "latitude", "longitude"]].drop_duplicates("code")
    print(f"[GADM2] Caricate {len(out):,} unità ADM2 uniche da {gadm_gpkg_path}")
    return out


def load_us_counties_points(counties_geojson_path: str, code_col: Optional[str] = None) -> pd.DataFrame:
    gdf = gpd.read_file(counties_geojson_path)
    candidates = [code_col] if code_col else [c for c in ["GEOID", "geoid", "FIPS", "fips"] if c in gdf.columns]
    if not candidates:
        raise ValueError("Non trovo una colonna codice per US counties (es. 'GEOID'). Passa code_col=...")
    cc = candidates[0]

    name_col = next((c for c in ["NAME", "NAMELSAD", "name"] if c in gdf.columns), None)

    gdf = gdf.rename(columns={cc: "code", (name_col if name_col else cc): "name"})
    gdf["code"] = gdf["code"].astype(str).str.zfill(5)  # GEOID a 5 cifre (es. 06091)
    gdf = _representative_points(gdf[["code", "name", "geometry"]].dropna(subset=["code"]))
    gdf["code"] = _normalize_location_code(gdf["code"])  # manteniamo 5 cifre

    out = gdf[["code", "name", "latitude", "longitude"]].drop_duplicates("code")
    return out


def load_gadm1_points(gadm_gpkg_path: str, layer: str = "gadm_410", code_col: str = "GID_1") -> pd.DataFrame:
    gdf = gpd.read_file(gadm_gpkg_path, layer=layer)
    if code_col not in gdf.columns:
        raise ValueError(f"Colonna {code_col!r} assente in {layer}. Colonne: {list(gdf.columns)}")
    name_col = "NAME_1" if "NAME_1" in gdf.columns else None

    gdf1 = gdf[~gdf[code_col].isna()].copy().rename(columns={code_col: "code"})
    if name_col:
        gdf1 = gdf1.rename(columns={name_col: "name"})
    gdf1 = gdf1.to_crs(4326)
    gdf1["geometry"] = gdf1.geometry.representative_point()
    gdf1["latitude"] = gdf1.geometry.y
    gdf1["longitude"] = gdf1.geometry.x
    gdf1["code"] = gdf1["code"].astype("string").str.strip().str.upper()  # es. "AGO.4_1"
    if "name" not in gdf1:
        gdf1["name"] = pd.NA

    return gdf1[["code", "name", "latitude", "longitude"]].drop_duplicates("code")


# ==========================
# Selezione target dal mapping (con GADM2)
# ==========================

def select_target_codes(
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    level_types: Iterable[str] = ("NUTS3", "GADM2", "GADM1", "COUNTY"),
    sci_required: Iterable[str] = ("user_loc", "fr_loc"),
    map_required: Iterable[str] = ("location_code", "level_type"),
) -> pd.DataFrame:
    _ensure_columns(df_sci, sci_required, "SCI")
    _ensure_columns(df_map, map_required, "Mapping")

    # Codici presenti nello SCI (user_loc ∪ fr_loc)
    sci_codes = pd.Index(_normalize_location_code(df_sci["user_loc"])) \
                  .append(pd.Index(_normalize_location_code(df_sci["fr_loc"]))) \
                  .unique()

    # Filtra mapping per i tipi desiderati e normalizza
    df_map2 = df_map.copy()
    df_map2["location_code"] = _normalize_location_code(df_map2["location_code"])
    df_map2["level_type"] = df_map2["level_type"].astype("string")

    wanted = {t.upper() for t in level_types}
    df_map2 = df_map2[df_map2["level_type"].str.upper().isin(wanted)]

    # Intersezione SCI ∩ mapping e deduplica
    target = (df_map2[df_map2["location_code"].isin(set(sci_codes))]
              .drop_duplicates(subset=["location_code", "level_type"])  # possono esistere più tipi
              .reset_index(drop=True))

    print(f"[INFO] Target codes selezionati: {len(target):,} "
          f"(tipi: {sorted(target['level_type'].str.upper().unique())})")
    return target


# ==========================
# Normalizzatori dalle chiavi di mapping -> chiavi geodati
# ==========================

def normalize_gadm1_mapping_code(code: Optional[str]) -> Optional[str]:
    """
    Esempi attesi nel mapping: 'BGD1', 'IND23', ...
    Output (GADM v4): 'ISO3.ADM1_1'  es. 'BGD.1_1'
    """
    if code is None or pd.isna(code):
        return None
    s = str(code).strip().upper()
    # Se già in formato GID_1
    if re.fullmatch(r"[A-Z]{3}\.\d+_1", s):
        return s
    m = re.fullmatch(r'([A-Z]{3})(\d+)', s)
    if not m:
        return None
    iso3, adm1 = m.groups()
    return f"{iso3}.{int(adm1)}_1"


def normalize_gadm2_mapping_code(code: Optional[str]) -> Optional[str]:
    """
    Converte varie forme in GADM v4 GID_2: 'ISO3.A.B_1' (es. 'AGO.4.7_1').
    Accetta anche forme come 'AGO4-7', 'AGO4.7', 'AGO-4-7', 'AGO4_7', ecc.
    """
    if code is None or pd.isna(code):
        return None
    s = str(code).strip().upper()
    # Già in formato GID_2
    if re.fullmatch(r"[A-Z]{3}\.\d+\.\d+_1", s):
        return s
    # Pattern flessibile: ISO3 + sep + ADM1 + sep + ADM2 (sep = . - _ o nessuno tra num)
    m = re.fullmatch(r"([A-Z]{3})[\.-_ ]?(\d+)[\.-_ ]?(\d+)", s)
    if not m:
        return None
    iso3, adm1, adm2 = m.groups()
    return f"{iso3}.{int(adm1)}.{int(adm2)}_1"


def normalize_county_mapping_code(code: Optional[str]) -> Optional[str]:
    """
    'USA06091' -> '06091' (GEOID a 5 cifre) per il join; il nodeLabel verrà poi ricostruito come 'USA' + GEOID.
    """
    if code is None or pd.isna(code):
        return None
    s = str(code).strip().upper()
    if s.startswith("USA"):
        s = s[3:]
    s = re.sub(r'\D', '', s)
    if len(s) == 5:
        return s
    if 1 <= len(s) <= 5:
        return s.zfill(5)
    return None


# ==========================
# Costruzione nodi per tipo (con name + DatasetDiOrigine + nodeLabel coerente)
# ==========================

def build_nodes_for_type_with_transform(
    target_codes: pd.DataFrame,
    type_name: str,
    source_df_points: pd.DataFrame,
    transform_fn=None
) -> pd.DataFrame:
    mask = target_codes["level_type"].str.upper() == type_name.upper()
    wanted = target_codes.loc[mask, ["location_code"]].copy()
    wanted["location_code"] = wanted["location_code"].astype("string").str.strip().str.upper()

    # Chiave 'code' per il join con i geodati
    if transform_fn is not None:
        wanted["code"] = wanted["location_code"].map(transform_fn)
    else:
        # Se il mapping già usa la stessa codifica dei geodati
        wanted["code"] = wanted["location_code"]

    wanted = wanted.dropna(subset=["code"]).drop_duplicates("code")

    pts = source_df_points.copy()
    pts["code"] = pts["code"].astype("string").str.strip().str.upper()

    out = wanted.merge(pts, on="code", how="inner")

    # nodeLabel desiderato per tipo
    type_upper = type_name.upper()
    if type_upper == "COUNTY":
        # Mostra 'USA' + GEOID a 5 cifre (es. USA06091)
        out["nodeLabel"] = "USA" + out["code"].astype(str).str.zfill(5)
    else:
        # Per NUTS3/GADM1/GADM2 la 'code' è già AL034 / AGO.4_1 / AGO.4.7_1
        out["nodeLabel"] = out["code"]

    out["DatasetDiOrigine"] = type_upper
    # Rinominare 'name' -> 'nodeName' se presente
    if "name" in out.columns:
        out = out.rename(columns={"name": "nodeName"})
    else:
        out["nodeName"] = pd.NA

    cols = ["nodeLabel", "nodeName", "latitude", "longitude", "DatasetDiOrigine"]
    return out[cols].drop_duplicates("nodeLabel")


def build_nodes_for_type(
    target_codes: pd.DataFrame,
    type_name: str,
    source_df_points: pd.DataFrame
) -> pd.DataFrame:
    return build_nodes_for_type_with_transform(
        target_codes=target_codes,
        type_name=type_name,
        source_df_points=source_df_points,
        transform_fn=None
    )


# ==========================
# Assemblaggio finale con priorità GADM2 > GADM1
# ==========================

def _gid1_base_from_gid1(gid1: str) -> Optional[str]:
    # 'AGO.4_1' -> 'AGO.4'
    if not isinstance(gid1, str):
        return None
    s = gid1.strip().upper()
    m = re.fullmatch(r"([A-Z]{3}\.\d+)_1", s)
    return m.group(1) if m else None


def _gid1_base_from_gid2(gid2: str) -> Optional[str]:
    # 'AGO.4.7_1' -> 'AGO.4'
    if not isinstance(gid2, str):
        return None
    s = gid2.strip().upper()
    m = re.fullmatch(r"([A-Z]{3}\.\d+)\.\d+_1", s)
    return m.group(1) if m else None


def drop_gadm1_if_gadm2_present(gadm1_nodes: pd.DataFrame, gadm2_nodes: pd.DataFrame) -> pd.DataFrame:
    if gadm1_nodes is None or len(gadm1_nodes) == 0:
        return gadm1_nodes
    if gadm2_nodes is None or len(gadm2_nodes) == 0:
        return gadm1_nodes

    gadm1 = gadm1_nodes.copy()
    gadm2 = gadm2_nodes.copy()

    gadm1["_gid1base"] = gadm1["nodeLabel"].map(_gid1_base_from_gid1)
    gadm2_bases: Set[str] = set(gadm2["nodeLabel"].map(_gid1_base_from_gid2).dropna().unique())

    # Mantieni solo GADM1 che NON hanno un GADM2 nello stesso ADM1
    keep_mask = ~gadm1["_gid1base"].isin(gadm2_bases)
    kept = gadm1.loc[keep_mask, [c for c in gadm1.columns if c != "_gid1base"]].copy()
    dropped = len(gadm1) - len(kept)
    if dropped:
        print(f"[INFO] Rimossi {dropped:,} nodi GADM1 perché coperti da GADM2 nello stesso ADM1")
    return kept


def assemble_node_list(
    nuts_nodes: Optional[pd.DataFrame] = None,
    gadm2_nodes: Optional[pd.DataFrame] = None,
    gadm1_nodes: Optional[pd.DataFrame] = None,
    county_nodes: Optional[pd.DataFrame] = None
) -> pd.DataFrame:
    # Applica priorità: NUTS3 (UE) + GADM2 + (GADM1 senza duplicati ADM1) + COUNTIES
    if gadm1_nodes is not None and gadm2_nodes is not None:
        gadm1_nodes = drop_gadm1_if_gadm2_present(gadm1_nodes, gadm2_nodes)

    frames = [df for df in [nuts_nodes, gadm2_nodes, gadm1_nodes, county_nodes] if df is not None and len(df)]
    if not frames:
        raise ValueError("Nessun nodo fornito.")

    nodes = pd.concat(frames, ignore_index=True)
    nodes = nodes.drop_duplicates("nodeLabel")
    nodes = nodes.sort_values("nodeLabel").reset_index(drop=True)
    nodes.insert(0, "nodeID", range(1, len(nodes) + 1))

    # Output finale richiesto: nodeID, nodeLabel, nodeName, latitude, longitude, DatasetDiOrigine
    return nodes[["nodeID", "nodeLabel", "nodeName", "latitude", "longitude", "DatasetDiOrigine"]]


# ==========================
# Ispezioni rapide dei layer (facoltative)
# ==========================
layers = fiona.listlayers(GADM_GPKG_PATH)
print(f"Layer trovati in {GADM_GPKG_PATH}:")
for i, lyr in enumerate(layers, 1):
    print(f"{i:>2}. {lyr}")

# Anteprime
try:
    nuts_sample = gpd.read_file(NUTS_GEOJSON_PATH, rows=5)
except TypeError:
    nuts_sample = gpd.read_file(NUTS_GEOJSON_PATH)
print("\nColonne disponibili in NUTS3 GeoJSON:")
print(list(nuts_sample.columns))
try:
    from IPython.display import display as _display
    _display(nuts_sample.head())
except Exception:
    pass

try:
    counties_sample = gpd.read_file(US_COUNTIES_PATH, rows=5)
except TypeError:
    counties_sample = gpd.read_file(US_COUNTIES_PATH)
print("\nColonne disponibili in US counties GeoJSON:")
print(list(counties_sample.columns))
try:
    _display(counties_sample.head())
except Exception:
    pass


# Diagnostica GADM (livello 2)
def inspect_gadm2(gpkg_path: str, layer: str = "gadm_410", prefer_code_cols=("GID_2","ID_2","GID2")):
    gdf = gpd.read_file(gpkg_path, layer=layer)
    print(f"[INFO] Caricato layer='{layer}' con {len(gdf):,} feature")
    print(f"[INFO] CRS: {gdf.crs}")
    print("[INFO] Colonne disponibili:")
    print(list(gdf.columns))

    cols_up = {c.upper(): c for c in gdf.columns}
    code_col = None
    for pref in prefer_code_cols:
        if pref in cols_up:
            code_col = cols_up[pref]
            break
    if code_col is None:
        candidates = [orig for up, orig in cols_up.items() if up.endswith("_2") or "GID" in up]
        if not candidates:
            raise ValueError("Non trovo una colonna codice per ADM2 (tipo 'GID_2' o 'ID_2').")
        code_col = candidates[0]

    print(f"[INFO] Colonna codice ADM2 individuata: {code_col!r}")
    gdf_adm2 = gdf[~gdf[code_col].isna()].copy()
    print(f"[INFO] Righe ADM2 (non-NaN su {code_col}): {len(gdf_adm2):,}")

    unique_codes = gdf_adm2[code_col].astype("string").str.strip().str.upper().nunique()
    print(f"[INFO] Codici ADM2 unici: {unique_codes:,}")

    geom_types = gdf_adm2.geom_type.value_counts().to_dict()
    print(f"[INFO] Geometrie (conteggio per tipo): {geom_types}")

    candidates_name = [c for c in ["NAME_0","NAME_1","NAME_2","GID_0","GID_1","GID_2"] if c in gdf_adm2.columns]
    show_cols = [code_col] + candidates_name
    show_cols = [c for c in dict.fromkeys(show_cols)]  # dedupe preservando ordine

    print("\n== Anteprima ADM2 (prime 5 righe) ==")
    try:
        _display(gdf_adm2[show_cols].head())
    except Exception:
        pass

    if any(c in gdf_adm2.columns for c in ["NAME_0","NAME_1","NAME_2","GID_0","GID_1","GID_2"]):
        print("\n== Missing count su colonne comuni ==")
        check_cols = [c for c in ["NAME_0","NAME_1","NAME_2","GID_0","GID_1","GID_2"] if c in gdf_adm2.columns]
        try:
            _display(gdf_adm2[check_cols].isna().sum().to_frame("missing"))
        except Exception:
            pass

    return gdf, gdf_adm2, code_col

# Esecuzione diagnostica (facoltativa, utile la prima volta)
gdf_full, gdf_adm2, gadm_code_col = inspect_gadm2(GADM_GPKG_PATH, layer="gadm_410")


# ==========================
# Pipeline NODI
# ==========================

# 0) Restrizione ai codici presenti in SCI + mapping (incluso GADM2)
target = select_target_codes(
    df_sci=df_sci,
    df_map=df_map,
    level_types=("NUTS3", "GADM2", "GADM1", "COUNTY")
)

# 1) Carico punti per ciascun tipo (con 'name')
nuts_pts    = load_nuts3_points(NUTS_GEOJSON_PATH)
county_pts  = load_us_counties_points(US_COUNTIES_PATH)
gadm1_pts   = load_gadm1_points(GADM_GPKG_PATH, layer="gadm_410", code_col="GID_1")
gadm2_pts   = load_gadm2_points(GADM_GPKG_PATH, layer="gadm_410", code_col="GID_2")

print("NUTS pts:", len(nuts_pts), "— US counties pts:", len(county_pts), "— GADM1 pts:", len(gadm1_pts), "— GADM2 pts:", len(gadm2_pts))

# 2) Join con eventuale trasformazione delle chiavi
nuts_nodes    = build_nodes_for_type_with_transform(target, "NUTS3",  nuts_pts,   transform_fn=None)
gadm2_nodes   = build_nodes_for_type_with_transform(target, "GADM2",  gadm2_pts,  transform_fn=normalize_gadm2_mapping_code)
gadm1_nodes   = build_nodes_for_type_with_transform(target, "GADM1",  gadm1_pts,  transform_fn=normalize_gadm1_mapping_code)
county_nodes  = build_nodes_for_type_with_transform(target, "COUNTY", county_pts, transform_fn=normalize_county_mapping_code)

print("NUTS3 selezionati:", len(nuts_nodes))
print("GADM2 selezionati:", len(gadm2_nodes))
print("GADM1 selezionati (prima del filtro):", len(gadm1_nodes))
print("COUNTY selezionati:", len(county_nodes))

# 3) Assemblaggio finale con priorità GADM2 > GADM1
node_list = assemble_node_list(nuts_nodes, gadm2_nodes, gadm1_nodes, county_nodes)

try:
    display(node_list.head())
except Exception:
    pass
print(f"Totale nodi: {len(node_list):,}")

# 4) Salvataggio
OUTPUT_CSV = "node_list.csv"
node_list.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Salvato: {OUTPUT_CSV} (colonne: {list(node_list.columns)})")


== SCI (TSV) – prime righe ==


,user_loc,fr_loc,scaled_sci
0,ABW,ABW,11264841.0
1,ABW,AGO1,38.0
2,ABW,AGO10,34.0
3,ABW,AGO11,32.0
4,ABW,AGO12,23.0



Schema:
user_loc      string[python]
fr_loc        string[python]
scaled_sci           float64
dtype: object

Numero righe: 63,824,121
== Mapping livelli – prime righe ==


,location_code,level_type
0,AND,country
1,ATG,country
2,ABW,country
3,BHS,country
4,BRB,country



Schema:
location_code    string[python]
level_type       string[python]
dtype: object

Numero righe: 8,008

== level_type value_counts ==


...

[INFO] Target codes selezionati: 7,936 (tipi: ['COUNTY', 'GADM1', 'GADM2', 'NUTS3'])


Skipping field geo_point_2d: unsupported OGR type: 3


[GADM2] Caricate 47,218 unità ADM2 uniche da data/gadm_410.gpkg
NUTS pts: 1522 — US counties pts: 3233 — GADM1 pts: 3662 — GADM2 pts: 47218
NUTS3 selezionati: 1522
GADM2 selezionati: 82
GADM1 selezionati (prima del filtro): 1738
COUNTY selezionati: 3225


,nodeID,nodeLabel,nodeName,latitude,longitude,DatasetDiOrigine
0,1,AGO.10_1,Huíla,-13.869244,15.319441,GADM1
1,2,AGO.11_1,Luanda,-8.789466,13.385529,GADM1
2,3,AGO.12_1,Lunda Norte,-8.303799,21.399342,GADM1
3,4,AGO.13_1,Lunda Sul,-10.888833,19.163756,GADM1
4,5,AGO.14_1,Malanje,-9.323313,15.771779,GADM1


Totale nodi: 6,567
✅ Salvato: node_list.csv (colonne: ['nodeID', 'nodeLabel', 'nodeName', 'latitude', 'longitude', 'DatasetDiOrigine'])


In [5]:
def summarize_fb_node_coverage(
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    node_list: pd.DataFrame,
    allowed_datasets: Iterable[str] = ("NUTS3", "GADM2", "GADM1", "COUNTY"),
    by_country: bool = True
) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    """
    Report di copertura dei NODI del Facebook SCI (user_loc ∪ fr_loc) rispetto ai nodi costruiti.
    Ritorna (df_type, df_country_or_None).
    """
    # Helpers locali
    def _detect_country_cols_local(df_map: pd.DataFrame) -> Tuple[Optional[str], Optional[str]]:
        iso_candidates = ["country_ISO3", "ISO3", "iso3", "COUNTRY_ISO3", "country_iso3"]
        name_candidates = ["country_name", "COUNTRY_NAME", "country", "COUNTRY", "NAME_0"]
        iso_col = next((c for c in iso_candidates if c in df_map.columns), None)
        name_col = next((c for c in name_candidates if c in df_map.columns), None)
        return iso_col, name_col

    def _node_label_from_mapping_row(level_type: str, location_code: str) -> Optional[str]:
        lt = (level_type or "").upper()
        if pd.isna(location_code):
            return None
        code = str(location_code).strip().upper()
        if lt == "NUTS3":
            return code
        if lt == "GADM1":
            return normalize_gadm1_mapping_code(code)
        if lt == "GADM2":
            return normalize_gadm2_mapping_code(code)
        if lt == "COUNTY":
            c = normalize_county_mapping_code(code)
            return None if c is None else ("USA" + str(c).zfill(5))
        return None

    allowed = {t.upper() for t in allowed_datasets}

    # 1) Codici FB unici (user_loc ∪ fr_loc)
    sci_codes = set(get_unique_location_codes(df_sci))

    # 2) Set di nodeLabel presenti per dataset nei nodi
    labels_by_ds: dict[str, set[str]] = {
        ds: set(node_list.loc[node_list["DatasetDiOrigine"].str.upper() == ds, "nodeLabel"].astype(str))
        for ds in ["NUTS3", "GADM2", "GADM1", "COUNTY"]
    }

    # 3) Colonne paese (se ci sono)
    iso_col, name_col = _detect_country_cols_local(df_map)

    rows_type = []
    rows_country = []

    for ds in ["NUTS3", "GADM2", "GADM1", "COUNTY"]:
        if ds not in allowed:
            continue

        mm = df_map[df_map["level_type"].str.upper() == ds].copy()
        mm = mm[mm["location_code"].isin(sci_codes)].copy()

        mm["nodeLabel"] = mm.apply(lambda r: _node_label_from_mapping_row(ds, r["location_code"]), axis=1)
        mm = mm.dropna(subset=["nodeLabel"]).drop_duplicates("location_code")

        fb_unique = mm["location_code"].nunique()
        matched = int(mm["nodeLabel"].isin(labels_by_ds.get(ds, set())).sum())

        rows_type.append({
            "dataset_type": ds,
            "fb_unique_nodes": int(fb_unique),
            "matched_nodes": matched,
            "coverage_pct": (matched / fb_unique * 100.0) if fb_unique else 0.0,
            "unmatched_nodes": int(fb_unique - matched),
        })

        if by_country and iso_col is not None:
            cols = ["location_code", iso_col]
            if name_col and name_col in df_map.columns and name_col != iso_col:
                cols.append(name_col)
            tmp = mm.merge(df_map[cols], on="location_code", how="left").rename(columns={iso_col: "country_ISO3"})
            if name_col and name_col in tmp.columns and name_col != iso_col:
                tmp = tmp.rename(columns={name_col: "country_name"})
            else:
                tmp["country_name"] = pd.NA
            tmp["is_matched"] = tmp["nodeLabel"].isin(labels_by_ds.get(ds, set()))

            for (iso3, cname), grp in tmp.groupby(["country_ISO3", "country_name"], dropna=False):
                fb_u = grp["location_code"].nunique()
                m_u = int(grp["is_matched"].sum())
                rows_country.append({
                    "country_ISO3": iso3,
                    "country_name": cname,
                    "dataset_type": ds,
                    "fb_unique_nodes": int(fb_u),
                    "matched_nodes": m_u,
                    "coverage_pct": (m_u / fb_u * 100.0) if fb_u else 0.0,
                    "unmatched_nodes": int(fb_u - m_u),
                })

    df_type = pd.DataFrame(rows_type).sort_values(["dataset_type"]).reset_index(drop=True)
    df_country = (pd.DataFrame(rows_country)
                    .sort_values(["dataset_type", "country_ISO3"])
                    .reset_index(drop=True)) if rows_country else None

    print("\n== Copertura FB → NODI per TIPO ==")
    try:
        from IPython.display import display as _display
        _display(df_type)
    except Exception:
        print(df_type)

    if df_country is not None:
        print("\n== Copertura FB → NODI per PAESE × TIPO (prime 20 righe) ==")
        try:
            _display(df_country.head(20))
        except Exception:
            print(df_country.head(20))

    return df_type, df_country


coverage_by_type, coverage_by_country = summarize_fb_node_coverage(
    df_sci=df_sci,
    df_map=df_map,
    node_list=node_list
)


== Copertura FB → NODI per TIPO ==


,dataset_type,fb_unique_nodes,matched_nodes,coverage_pct,unmatched_nodes
0,COUNTY,3225,3225,100.000000,0
1,GADM1,1819,1738,95.547004,81
2,GADM2,1370,82,5.985401,1288
3,NUTS3,1522,1522,100.000000,0


In [7]:
# ==========================
# EDGE LIST: generale e per-paese (ISO3 auto da NUTS/GADM/USA)
# ==========================

import os
import re
from typing import Iterable, Dict, Optional, Tuple, List, Set

# --- Mappa NUTS alpha-2 -> ISO3 ricavata dal GeoJSON NUTS ---
def build_nuts_a2_to_iso3_map(nuts_geojson_path: str) -> Dict[str, str]:
    gdf = gpd.read_file(nuts_geojson_path)
    # CNTR_CODE è alpha-2 (con le eccezioni NUTS: UK, EL)
    if "CNTR_CODE" not in gdf.columns:
        raise ValueError("Nel GeoJSON NUTS manca la colonna 'CNTR_CODE'.")
    a2_vals = (
        gdf["CNTR_CODE"]
        .dropna().astype(str).str.upper().unique().tolist()
    )
    mapping: Dict[str, str] = {}
    try:
        import pycountry
        for a2 in a2_vals:
            a2_norm = "GB" if a2 == "UK" else ("GR" if a2 == "EL" else a2)
            c = pycountry.countries.get(alpha_2=a2_norm)
            if c and hasattr(c, "alpha_3"):
                mapping[a2] = c.alpha_3
    except Exception:
        # fallback minimo se pycountry non disponibile
        repl = {"UK": "GBR", "EL": "GRC"}
        for a2 in a2_vals:
            if a2 in repl:
                mapping[a2] = repl[a2]
    if not mapping:
        print("[WARN] Impossibile costruire mapping NUTS alpha2 -> ISO3. Installa 'pycountry'.")
    return mapping

# Istanzia una volta (usa la tua variabile globale NUTS_GEOJSON_PATH)
NUTS_A2_TO_ISO3: Dict[str, str] = build_nuts_a2_to_iso3_map(NUTS_GEOJSON_PATH)

def iso3_from_node_label(nodeLabel: str, dataset: str) -> Optional[str]:
    """Inferisce ISO3 dal nodeLabel a seconda del dataset."""
    if not isinstance(nodeLabel, str):
        return None
    ds = (dataset or "").upper()
    if ds in ("GADM1", "GADM2"):
        m = re.match(r"^([A-Z]{3})\.", nodeLabel)
        return m.group(1) if m else None
    if ds == "COUNTY":
        return "USA"
    if ds == "NUTS3":
        a2 = nodeLabel[:2].upper()
        return NUTS_A2_TO_ISO3.get(a2)
    return None

def country_name_from_iso3(iso3: Optional[str]) -> Optional[str]:
    """Ritorna un country name leggibile da ISO3 (se 'pycountry' è installato)."""
    if iso3 is None or pd.isna(iso3):
        return None
    try:
        import pycountry
        c = pycountry.countries.get(alpha_3=str(iso3))
        if c:
            return getattr(c, "common_name", None) or getattr(c, "name", None)
    except Exception:
        pass
    return None

def _detect_country_cols(df_map: pd.DataFrame) -> Tuple[Optional[str], Optional[str]]:
    """Versione permissiva: se non ci sono colonne ISO3/nome nel mapping, restituisce (None, None)."""
    iso_candidates = ["country_ISO3", "ISO3", "iso3", "COUNTRY_ISO3", "country_iso3"]
    name_candidates = ["country_name", "COUNTRY_NAME", "country", "COUNTRY", "NAME_0"]
    iso_col = next((c for c in iso_candidates if c in df_map.columns), None)
    name_col = next((c for c in name_candidates if c in df_map.columns), None)
    return iso_col, name_col

def build_loc2node_mapping(
    nodes: pd.DataFrame,
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    allowed_datasets: Iterable[str] = ("NUTS3", "GADM2", "GADM1", "COUNTY")
) -> pd.DataFrame:
    """
    location_code (SCI) -> (nodeID, nodeLabel, DatasetDiOrigine, country_ISO3, country_name)
    ISO3 viene:
      - da df_map se presente; ALTRIMENTI
      - inferito da nodeLabel + dataset (NUTS via CNTR_CODE; GADM dal prefisso; COUNTY=USA).
    """
    allowed = {t.upper() for t in allowed_datasets}
    nodes_ok = nodes[nodes["DatasetDiOrigine"].str.upper().isin(allowed)].copy()

    # target: codici SCI ∩ mapping, limitati ai tipi ammessi
    target = select_target_codes(df_sci=df_sci, df_map=df_map, level_types=allowed)

    # Costruisci nodeLabel atteso per tipo (come nei nodi)
    parts = []
    if "NUTS3" in allowed:
        t = target[target["level_type"].str.upper() == "NUTS3"].copy()
        t["code"] = t["location_code"]
        t["nodeLabel"] = t["code"]
        parts.append(t)
    if "GADM2" in allowed:
        t = target[target["level_type"].str.upper() == "GADM2"].copy()
        t["code"] = t["location_code"].map(normalize_gadm2_mapping_code)
        t["nodeLabel"] = t["code"]
        parts.append(t)
    if "GADM1" in allowed:
        t = target[target["level_type"].str.upper() == "GADM1"].copy()
        t["code"] = t["location_code"].map(normalize_gadm1_mapping_code)
        t["nodeLabel"] = t["code"]
        parts.append(t)
    if "COUNTY" in allowed:
        t = target[target["level_type"].str.upper() == "COUNTY"].copy()
        t["code"] = t["location_code"].map(normalize_county_mapping_code)
        t["nodeLabel"] = "USA" + t["code"].astype(str).str.zfill(5)
        parts.append(t)

    if not parts:
        raise ValueError("Nessun dataset consentito per costruire il mapping loc->node.")

    mapping_raw = (pd.concat(parts, ignore_index=True)
                     .dropna(subset=["nodeLabel"])
                     .drop_duplicates(["location_code", "nodeLabel"]))

    # Allinea ai nodi effettivi
    loc2node = mapping_raw.merge(
        nodes_ok[["nodeID", "nodeLabel", "DatasetDiOrigine"]],
        on="nodeLabel", how="inner"
    )

    # --- Paese: usa df_map se disponibile, altrimenti inferisci ---
    iso_col, name_col = _detect_country_cols(df_map)
    if iso_col is not None:
        loc2country = df_map[["location_code", iso_col]].copy().rename(columns={iso_col: "country_ISO3"})
        if name_col and name_col in df_map.columns and name_col != iso_col:
            loc2country["country_name"] = df_map[name_col]
        else:
            loc2country["country_name"] = pd.NA
        loc2node = loc2node.merge(loc2country, on="location_code", how="left")
    else:
        loc2node["country_ISO3"] = pd.NA
        loc2node["country_name"] = pd.NA

    # Fallback ISO3 per i mancanti: inferisci da nodeLabel+dataset
    miss_iso = loc2node["country_ISO3"].isna()
    if miss_iso.any():
        loc2node.loc[miss_iso, "country_ISO3"] = loc2node.loc[miss_iso].apply(
            lambda r: iso3_from_node_label(r["nodeLabel"], r["DatasetDiOrigine"]), axis=1
        ).astype("string")

    # Fallback country_name se mancante: prova via pycountry
    miss_name = loc2node["country_name"].isna() if "country_name" in loc2node.columns else pd.Series(True, index=loc2node.index)
    if miss_name.any():
        loc2node.loc[miss_name, "country_name"] = loc2node.loc[miss_name, "country_ISO3"].map(country_name_from_iso3)

    # Deduplica per location_code
    loc2node = (loc2node
                .sort_values(["location_code", "DatasetDiOrigine"])  # priorità stabilita a monte dai nodi
                .drop_duplicates("location_code"))

    assert loc2node["nodeID"].isna().sum() == 0, "Mapping loc->node con nodeID mancanti"
    return loc2node[["location_code", "nodeID", "nodeLabel", "DatasetDiOrigine", "country_ISO3", "country_name"]]

def build_edge_lists(
    df_sci: pd.DataFrame,
    df_map: pd.DataFrame,
    nodes: pd.DataFrame,
    allowed_datasets: Iterable[str] = ("NUTS3", "GADM2", "GADM1", "COUNTY"),
    output_dir: str = ".",
    general_filename: str = "edges_all.csv",
    per_country: bool = True,
    country_column_mode: str = "from"  # 'from' | 'to' | 'common_or_null'
) -> Tuple[pd.DataFrame, Dict[str, pd.DataFrame]]:
    """
    Output:
      - edges_all.csv: (nodeID_from,nodeID_to,country_name,country_ISO3,weight)
      - edges_by_country/edges_<ISO3>.csv (solo intra-paese)
    """
    os.makedirs(output_dir, exist_ok=True)

    # Mapping location_code (SCI) -> nodo + paese
    loc2node = build_loc2node_mapping(nodes, df_sci, df_map, allowed_datasets)

    # Join SCI con mapping from/to
    sci = df_sci[["user_loc", "fr_loc", "scaled_sci"]].copy()
    sci["user_loc"] = _normalize_location_code(sci["user_loc"])
    sci["fr_loc"]   = _normalize_location_code(sci["fr_loc"])

    left  = loc2node.add_prefix("from_")
    right = loc2node.add_prefix("to_")

    e = (sci
         .merge(left,  left_on="user_loc", right_on="from_location_code", how="inner")
         .merge(right, left_on="fr_loc",  right_on="to_location_code",   how="inner"))

    # Colonne paese da esporre
    if country_column_mode == "to":
        country_iso  = e["to_country_ISO3"]
        country_name = e["to_country_name"]
    elif country_column_mode == "common_or_null":
        same = e["from_country_ISO3"].notna() & (e["from_country_ISO3"] == e["to_country_ISO3"])
        country_iso  = e["from_country_ISO3"].where(same)
        country_name = e["from_country_name"].where(same)
    else:  # default: 'from'
        country_iso  = e["from_country_ISO3"]
        country_name = e["from_country_name"]

    edges_all = pd.DataFrame({
        "nodeID_from": e["from_nodeID"].astype(int),
        "nodeID_to":   e["to_nodeID"].astype(int),
        "country_name": country_name.astype("string"),
        "country_ISO3": country_iso.astype("string"),
        "weight":      e["scaled_sci"].astype(float),
    })

    # Salva generale
    out_all = os.path.join(output_dir, general_filename)
    edges_all.to_csv(out_all, index=False)
    print(f"✅ Salvato edges generale: {out_all} (righe: {len(edges_all):,})")

    # Edges intra-paese
    edges_by_iso: Dict[str, pd.DataFrame] = {}
    if per_country:
        intra_mask = e["from_country_ISO3"].notna() & (e["from_country_ISO3"] == e["to_country_ISO3"])
        intra = e.loc[intra_mask].copy()
        if len(intra):
            by_dir = os.path.join(output_dir, "edges_by_country")
            os.makedirs(by_dir, exist_ok=True)
            intra["country_ISO3"] = intra["from_country_ISO3"].astype(str)
            intra["country_name"] = intra["from_country_name"]

            for iso3, grp in intra.groupby("country_ISO3"):
                df_iso = pd.DataFrame({
                    "nodeID_from": grp["from_nodeID"].astype(int),
                    "nodeID_to":   grp["to_nodeID"].astype(int),
                    "country_name": grp["country_name"].astype("string"),
                    "country_ISO3": grp["country_ISO3"].astype("string"),
                    "weight":       grp["scaled_sci"].astype(float)
                })
                edges_by_iso[iso3] = df_iso
                file_iso = os.path.join(by_dir, f"edges_{iso3}.csv")
                df_iso.to_csv(file_iso, index=False)
            print(f"✅ Salvati {len(edges_by_iso)} file intra-paese in: {by_dir}")
        else:
            print("[INFO] Nessun edge intra-paese trovato con i filtri correnti.")

    return edges_all, edges_by_iso

# ==========================
# ESEMPIO DI ESECUZIONE
# ==========================
ALLOWED_DATASETS = ("NUTS3", "GADM2", "GADM1", "COUNTY")

df_edges_all, edges_per_country = build_edge_lists(
    df_sci=df_sci,
    df_map=df_map,
    nodes=node_list,
    allowed_datasets=ALLOWED_DATASETS,
    output_dir=".",
    general_filename="edges_all.csv",
    per_country=True,
    country_column_mode="from"
)


[INFO] Target codes selezionati: 7,936 (tipi: ['COUNTY', 'GADM1', 'GADM2', 'NUTS3'])
✅ Salvato edges generale: ./edges_all.csv (righe: 43,125,489)
✅ Salvati 145 file intra-paese in: ./edges_by_country


In [8]:
import os, pandas as pd

os.makedirs("cache", exist_ok=True)

# variabili principali già create dal tuo script
names = [
    "df_sci","df_map",
    "nuts_pts","gadm2_pts","gadm1_pts","county_pts",
    "target",
    "nuts_nodes","gadm2_nodes","gadm1_nodes","county_nodes",
    "node_list",
    "df_edges_all","edges_per_country",
    "coverage_by_type","coverage_by_country"
]

# salva se esistono nel namespace
for n in names:
    if n in globals():
        pd.to_pickle(globals()[n], f"cache/{n}.pkl")
        print(f"✅ Salvato {n} → cache/{n}.pkl")
    else:
        print(f"[WARN] Variabile non trovata: {n}")

# comodi extra ------------------------------------------

# mappa rapida nodeLabel -> nodeID
if "node_list" in globals():
    node_id_map = node_list.set_index("nodeLabel")["nodeID"].to_dict()
    pd.to_pickle(node_id_map, "cache/node_id_map.pkl")
    print("✅ Salvato node_id_map → cache/node_id_map.pkl")

# mapping location_code (SCI) -> nodo (se vuoi riusarli senza rifare join)
try:
    ALLOWED_DATASETS = ("NUTS3","GADM2","GADM1","COUNTY")
    loc2node = build_loc2node_mapping(node_list, df_sci, df_map, ALLOWED_DATASETS)
    pd.to_pickle(loc2node, "cache/loc2node.pkl")
    print("✅ Salvato loc2node → cache/loc2node.pkl")
except Exception as e:
    print(f"[WARN] loc2node non salvato: {e}")


✅ Salvato df_sci → cache/df_sci.pkl
✅ Salvato df_map → cache/df_map.pkl
✅ Salvato nuts_pts → cache/nuts_pts.pkl
✅ Salvato gadm2_pts → cache/gadm2_pts.pkl
✅ Salvato gadm1_pts → cache/gadm1_pts.pkl
✅ Salvato county_pts → cache/county_pts.pkl
✅ Salvato target → cache/target.pkl
✅ Salvato nuts_nodes → cache/nuts_nodes.pkl
✅ Salvato gadm2_nodes → cache/gadm2_nodes.pkl
✅ Salvato gadm1_nodes → cache/gadm1_nodes.pkl
✅ Salvato county_nodes → cache/county_nodes.pkl
✅ Salvato node_list → cache/node_list.pkl
✅ Salvato df_edges_all → cache/df_edges_all.pkl
✅ Salvato edges_per_country → cache/edges_per_country.pkl
✅ Salvato coverage_by_type → cache/coverage_by_type.pkl
✅ Salvato coverage_by_country → cache/coverage_by_country.pkl
✅ Salvato node_id_map → cache/node_id_map.pkl
[INFO] Target codes selezionati: 7,936 (tipi: ['COUNTY', 'GADM1', 'GADM2', 'NUTS3'])
✅ Salvato loc2node → cache/loc2node.pkl
